# Manual Testing Notebook (Enterprise Edition)

This notebook verifies the new features: Real Data Integration (CSV), RAG (Regulatory Chat), and the Validation Pipeline.

In [ ]:
import os
import sys
import yaml
import pandas as pd

# Ensure we can import from src
sys.path.append(os.path.abspath('..'))

from src.data_loader_module import get_data_loader, CSVDataLoader, SyntheticDataLoader
from src.calculations import CreditRiskFormulas
from src.pipeline import ValidationPipeline
from src.rag_engine import RagEngine

## 1. Test Data Loading (CSV vs Synthetic)

In [ ]:
# Test Synthetic First
print("Testing Synthetic Loader:")
synth_loader = SyntheticDataLoader()
print("AUC Data Head:")
display(synth_loader.load_auc_data().head())

In [ ]:
# Test CSV Loader (Expect Error if files missing)
print("Testing CSV Loader:")
try:
    csv_loader = CSVDataLoader("../metrics_from_client")
    display(csv_loader.load_auc_data().head())
except Exception as e:
    print(f"CSV Load Failed as expected (if no files): {e}")

# Note: To pass this test, put valid CSVs in metrics_from_client

## 2. Test RAG Engine

In [ ]:
rag = RagEngine("../config.yaml")
print("Ingesting Documents...")
# Ensure you have at least one PDF in ./documents to test this
status = rag.ingest_documents()
print(status)

In [ ]:
print("Testing Query...")
response = rag.query_knowledge_base(
    question="What is the policy on AUC drop?",
    stats_context="Current AUC dropped by 0.05."
)
print("RAG Response:")
print(response)

## 3. Test Full Pipeline with Real Data

In [ ]:
# Load Config
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Initialize Pipeline
pipeline = ValidationPipeline("../config.yaml")
calculator = CreditRiskFormulas(config)

# Load Data (Switch to Synthetic for reliable test run if CSVs missing)
loader = SyntheticDataLoader()
df_auc = loader.load_auc_data()
df_calib = loader.load_calibration_data()
df_scores = loader.load_score_data()

# Calculate Stats
auc_stats = calculator.evaluate_auc(df_auc)
chi_stats = calculator.calculate_chi_square(df_calib)
binomial_results = calculator.calculate_binomial_test(df_calib)
jeffrey_results = calculator.calculate_jeffrey_test(df_calib)
ttest_stats = calculator.calculate_ttest(df_scores)

# Run Pipeline
print("Running Pipeline...")
result = pipeline.run(
    auc_stats, chi_stats, binomial_results, jeffrey_results, ttest_stats
)

print(f"Status: {result['status']}")
print(f"Score: {result['score']}")